## Model Management and Monitoring

**Author**: Thodoris Petropoulos

**Label**: Model Deployment

### Scope

The scope of this notebook is to provide instructions on how to do manage models through the Python API. This includes deployment, replacement, deletion, and monitoring capabilities.

### Background

Deployment is the central hub for users to deploy, manage and monitor their models. The following commands can be used to manage deployments.

### Requirements

- Python version 3.7.3
-  DataRobot API version 2.19.0. 
Small adjustments might be needed depending on the Python version and DataRobot API version you are using.

Full documentation of the Python package can be found here: https://datarobot-public-api-client.readthedocs-hosted.com/en/

It is assumed you already have a DataRobot <code>Project</code> object and a DataRobot <code>Model </code> object.

#### Import Libraries

In [ ]:
import datarobot as dr

#### Create a Deployment

When creating a new deployment, a DataRobot model_id and label must be provided. A description can be optionally provided to document the purpose of the deployment.

The default prediction server is used when making predictions against the deployment, and is a requirement for creating a deployment on DataRobot cloud. For on-prem installations, a user must not provide a default prediction server and a pre-configured prediction server will be used instead. Refer to ListPredictionServers for more information on retrieving available prediction servers.

In [ ]:
prediction_server = dr.PredictionServer.list()[0]

deployment = dr.Deployment.create_from_learning_model(
    model.id, label='New Deployment', description='A new deployment',
    default_prediction_server_id=prediction_server.id)
deployment

### List Deployments
Use the following command to list deployments that are available.

In [ ]:
deployments = dr.Deployment.list()
deployments

### Retrieve a Deployment
It is possible to retrieve a single deployment with its identifier, rather than list all deployments.

In [ ]:
deployment = dr.Deployment.get(deployment_id='') #Fill in the id of your own deployment id

#### Update a Deployment
Deployment’s label and description can be updated.

In [ ]:
deployment = dr.Deployment.get(deployment_id='YOUR_DEPLOYMENT_ID') #Fill in the id of your own deployment id
deployment.update(label='new label')

#### Delete a Deployment
To mark a deployment as deleted, use the following command.

In [ ]:
deployment = dr.Deployment.get(deployment_id='YOUR_DEPLOYMENT_ID') #Fill in the id of your own deployment id
deployment.delete()

#### Model Replacement
The model of a deployment can be replaced effortlessly with zero interruption of predictions.
Model replacement is an asynchronous process, which means there are some preparatory works to complete before the process is fully finished. However, predictions made against this deployment will start using the new model as soon as you initiate the process. The <code>replace_model()</code> function won’t return until this asynchronous process is fully finished.
Alongside the identifier of the new model, a reason is also required. The reason is stored in model history of the deployment for bookkeeping purpose. An enum <code>MODEL_REPLACEMENT_REASON</code> is provided for convenience, all possible values are documented below:


- MODEL_REPLACEMENT_REASON.ACCURACY
- MODEL_REPLACEMENT_REASON.DATA_DRIFT
- MODEL_REPLACEMENT_REASON.ERRORS
- MODEL_REPLACEMENT_REASON.SCHEDULED_REFRESH
- MODEL_REPLACEMENT_REASON.SCORING_SPEED
- MODEL_REPLACEMENT_REASON.OTHER

Here is an example of model replacement:

In [ ]:
deployment = Deployment.get(deployment_id='YOUR_DEPLOYMENT_ID') #Fill in the id of your own deployment id
deployment.model['id'], deployment.model['type']

deployment.replace_model('YOUR_MODEL_ID', MODEL_REPLACEMENT_REASON.ACCURACY) #Fill in with id of model
deployment.model['id'], deployment.model['type']

#### Validation - Before Replacement
Before initiating the model replacement request, it is usually a good idea to use the <code>validate_replacement_model()</code> function to validate if the new model can be used as a replacement.

The <code>validate_replacement_model()</code> function returns the validation status, a message and a checks dictionary. If the status is ‘passing’ or ‘warning’, use <code>replace_model()</code> to perform model the replacement. If status is ‘failing’, refer to the checks dictionary for more details on why the new model cannot be used as a replacement.


In [ ]:
deployment = dr.Deployment.get(deployment_id='YOUR_DEPLOYMENT_ID')
status, message, checks = deployment.validate_replacement_model(new_model_id=model.id)
status

# `checks` can be inspected for detail, showing two examples here:
checks['target']
checks['permission']

#### Monitoring
Deployment monitoring can be categorized into several area of concerns:

- Service Stats & Service Stats Over Time
- Accuracy & Accuracy Over Time

With a Deployment object, get functions are provided to allow querying of the monitoring data. Alternatively, it is also possible to retrieve monitoring data directly using a deployment ID. For example:


In [ ]:
from datarobot.models import Deployment, ServiceStats

deployment_id = 'YOUR_DEPLOYMENT_ID'

# call `get` functions on a `Deployment` object
deployment = Deployment.get(deployment_id)
service_stats = deployment.get_service_stats()

# directly fetch without a `Deployment` object
service_stats = ServiceStats.get(deployment_id)

When querying monitoring data, a start and end time can be optionally provided, will accept either a datetime object or a string. Note that only top of the hour datetimes are accepted, for example: 2019-08-01T00:00:00Z. By default, the end time of the query will be the next top of the hour, the start time will be 7 days before the end time.

In the over time variants, an optional bucket_size can be provided to specify the resolution of time buckets. For example, if start time is 2019-08-01T00:00:00Z, end time is 2019-08-02T00:00:00Z and bucket_size is T1H, then 24 time buckets will be generated, each providing data calculated over one hour. Use construct_duration_string() to help construct a bucket size string.